In [4]:
def createdic(start,end):
    dic={}
    for i in range(int(start),int(end)):
        dic[i]='https://en.wikipedia.org/wiki/{}_Mosconi_Cup'.format(i)
    return(dic)

In [49]:
def createtables(url):
    import requests
    from lxml import html
    page = requests.get(url)
    tree = html.fromstring(page.text)
    tables = tree.xpath('//table')
    tables=[table for table in tables if(table[0].text_content().replace('\n','')=='Results')]
    return(tables)

In [18]:
def cdf(year,tables,df):
    import pandas as pd
    import re
    l=[]
    col=['Format','European_player','European_score','American_score','American_player']
    pattern = re.compile(r"\s?(\w+)\s(.+)\s([0-9])[–]([0-9])\s?\w+\s?(.+)\s")
    for i,table in enumerate(tables):
        for j in range(len(table)-2):
            try:
                element=tables[i][j].text_content().replace("\n", " ")
                match = re.match(pattern, element)
                if match:
                    l.append(list(match.groups()))
            except:
                pass
    if df.empty:
        df=pd.DataFrame(l,columns=col)
        df['Year']=year
        df.index.rename('Match',inplace=True)
        df.reset_index(inplace=True)
        df.set_index(['Year','Match'],inplace=True)
    else:
        new = pd.DataFrame(l, columns = col)
        new['Year']=year
        new.index.rename('Match',inplace=True)
        new.reset_index(inplace=True)
        new.set_index(['Year','Match'],inplace=True)
        frames = (df,new)
        df = pd.concat(frames)
    return(df)

In [51]:
def singlemain():
    import pandas as pd
    dic = createdic(1998,2014)
    year = 1998
    link = dic[year]
    df = pd.DataFrame()    
    table = createtables(link)
    df = cdf(year,table,df)
    return(df)

In [35]:
def newmain():
    import pandas as pd
    dic = createdic(1998,2014)
    df = pd.DataFrame()
    for year,link in dic.items():
        tables = createtables(link)
        df = icdf(year,tables,df)
    return(df)

In [170]:
df = newmain()

Missing 2014 data. Can get it from: http://www.matchroompool.com/mosconi-cup/mosconi-cup-xxi/#fixtures
compare stats here:
http://forums.azbilliards.com/showthread.php?t=439775&page=4

In [176]:
import re
import pandas as pd
l=[]
col=['Format','European_player','European_score','American_score','American_player']

with open('manual_scrape_cleaned_2014.txt','r') as rf:
    for line in rf:
        match = re.match(r"(\w+)\s(.+)\s([0-9])\s?[-]\s?([0-9])\s?(.+)", line)
        if match:
            l.append(list(match.groups()))
        else:
            print('fail')
df2014
# df2014=pd.DataFrame(l,columns=col)
# df2014['Europe_won']=df.European_score>df.American_score
# df2014

,Format,European_player,European_score,American_score,American_player
0,Teams,Team Europe,5,1,Team USA
1,Doubles,Darren Appleton & Karl Boyes,3,5,Corey Deuel & Justin Hall
2,Singles,Mark Gray,3,5,John Schmidt
3,Doubles,Niels Feijen & Nikos Ekonomopoulos,5,1,Shane Van Boening & Justin Bergman
4,Singles,Darren Appleton,5,3,Shane Van Boening
5,Doubles,Niels Feijen & Mark Gray,2,5,Justin Hall & Justin Bergman
6,Singles,Karl Boyes,5,2,Corey Deuel
7,Doubles,Darren Appleton & Nikos Ekonomopoulos,5,2,Shane Van Boening & John Schmidt
8,Singles,Niels Feijen,5,3,Justin Bergman
9,Doubles,Darren Appleton & Mark Gray,4,5,Shane Van Boening & Corey Deuel


In [177]:
frames=[df,df2014]
df=pd.concat(frames)

2015 Data is backwards, need to fix the order (european player should be first)

In [191]:
def createdf_bw(tables):
    import re
    l=[]
    col=['Format','European_player','European_score','American_score','American_player']

    for i in range(len(tables)+1):
        for j in range(1,6):
            try:
                element=tables[i][j].text_content().replace("\n", " ")
                match = re.match(r"\s?(\w+)\s(.+)\s([0-9])[–]([0-9])\s?\w+\s?(.+)\s", element)
                if match:
                    lst=[match.group(1), match.group(5), match.group(4), match.group(3), match.group(2)]
                    l.append(lst)
            except:
                pass
    print(l)
    df=pd.DataFrame(l,columns=col)
    return(df)

In [192]:
def main_bw():
    from lxml import html
    import requests
    import pandas as pd
    df = pd.DataFrame()
    links = createlinks('2015','2016')
    for url in links:
        tables = createtables(url)
        tl = tablelengths(tables)
        newdf = createdf_bw(tables) 
        df=df.append(newdf)
    return(df)

In [193]:
df2015= main_bw()

[['Teams', 'Team Europe', '4', '5', 'Team USA'], ['Doubles', 'Karl Boyes Nick van den Berg', '5', '4', 'Corey Deuel Mike Dechaine'], ['Singles', 'Niels Feijen', '5', '2', 'Justin Bergman'], ['Doubles', 'Darren Appleton Albin Ouschan', '1', '5', 'Shane Van Boening Skyler Woodward'], ['Singles', 'Darren Appleton', '5', '1', 'Shane Van Boening'], ['Doubles', 'Niels Feijen Nick van den Berg', '3', '5', 'Justin Bergman Skyler Woodward'], ['Singles', 'Albin Ouschan', '5', '3', 'Mike Dechaine'], ['Doubles', 'Darren Appleton Karl Boyes', '5', '2', 'Shane Van Boening Corey Deuel'], ['Singles', 'Karl Boyes', '5', '3', 'Skyler Woodward'], ['Doubles', 'Darren Appleton Nick van den Berg', '4', '5', 'Shane Van Boening Mike Dechaine'], ['Doubles', 'Niels Feijen Albin Ouschan', '5', '1', 'Justin Bergman Corey Deuel'], ['Singles', 'Nick van den Berg', '4', '5', 'Corey Deuel'], ['Doubles', 'Karl Boyes Albin Ouschan', '5', '2', 'Mike Dechaine Skyler Woodward'], ['Singles', 'Niels Feijen', '5', '4', 'Shan

In [182]:
frames=[df,df2015]
df=pd.concat(frames)

In [185]:
pkl = 'new_inc_2015_fixed_again'
pd.to_pickle(df,pkl)

Next steps: Rescrape the data. 

1. Need to include the year as a multiindex and concatenate the frames per year.(done)
1. Need to reconfigure the number of rows to possibly extend beyond 5. Have to dynamically learn the number of rows in the    given table and then use that number instead of hard coding it. (done)
3. Need to collect the 'team' data each year. I.e. what players are on each team so that I can do analysis on the 'team' matches and include that in the final presentations.
4. Scrape location each year and form dictionary b/w year and location